## 连接Ansys功能
首先创建一个控制的实例，使用`Ansys.connect_to_proj()`函数

In [1]:
from power_toys.components.ansys import Ansys
# 填入对应的项目名和design名字，如果没有已经打开的项目，将新建
m3d = Ansys.connect_to_proj('Project2','Maxwell3DDesign1',desktop_version="2022.1")

PyAEDT INFO: StdOut has been enabled
PyAEDT INFO: using existing logger.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path D:\Program Files\Ansys22\v221\Win64.
PyAEDT INFO: Launching AEDT with module PythonNET.
PyAEDT INFO: AEDT 2022.1 Started with process ID 10608.
PyAEDT INFO: pyaedt v0.6.81
PyAEDT INFO: Python version 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
PyAEDT INFO: AEDT 2022.1.0 Build Date 2021-11-22 22:18:25
PyAEDT INFO: Project Project2 set to active.
PyAEDT INFO: Aedt Objects initialized
PyAEDT INFO: Variable Manager initialized
PyAEDT INFO: Design Loaded
PyAEDT INFO: Materials Loaded
PyAEDT INFO: aedt file load time 0.03194689750671387


## 建模

In [2]:
from power_toys.components.ansys import Ansys
# 导入ferrite材质
Ansys.import_ferrite(m3d)
# 先删掉所有的模型
Ansys.delete_all(m3d)

# 配置variable参数
m3d['_r'] = '6.5mm'
m3d['_k'] = '600percent'
m3d['_w'] = '11mm'
# 创建strip形状
Ansys.create_strip(m3d,r='_r',k='_k',matname='ferrite_loss',name='left_leg')
Ansys.create_strip(m3d,position=[0,'2*_r+2*_w',0],r='_r',k='_k',matname='ferrite_loss',name='right_leg')
# 创建winding形状
Ansys.create_winding(m3d,position=[0,'0mm','1mm'],r='_r',k='_k',w='_w',name='left_winding')
Ansys.create_winding(m3d,position=[0,'2*_r+2*_w','1mm'],r='_r',k='_k',w='_w',name='right_winding')
# 创建I片
Ansys.create_I_plate(m3d,position=['-_r','-_r','3.5mm'],r='_r',k='_k',w='_w',height='8mm',matname='ferrite_loss',name='top_plate')
Ansys.create_I_plate(m3d,position=['-_r','-_r','-0.5mm'],r='_r',k='_k',w='_w',height='-8mm',matname='ferrite_loss',name='bottom_plate')

# 创建两个面
# Ansys.create_face_from_volume(m3d,'top_plate','top','top_plate_top')
# Ansys.create_face_from_volume(m3d,'top_plate','bottom','top_plate_bottom')

m3d.modeler.create_region(200)

PyAEDT INFO: Successfully loaded project materials !


PyAEDT WARNING: Material ferrite_loss already exists. Renaming to ferrite_loss_4LLEDL
PyAEDT INFO: Deleted 1 Objects: Region.
PyAEDT INFO: Deleted 1 Objects: left_leg.
PyAEDT INFO: Deleted 1 Objects: right_leg.
PyAEDT INFO: Deleted 1 Objects: left_winding.
PyAEDT INFO: Deleted 1 Objects: right_winding.
PyAEDT INFO: Deleted 1 Objects: top_plate.
PyAEDT INFO: Deleted 1 Objects: bottom_plate.
PyAEDT INFO: Deleted 1 Objects: left_winding_cross.
PyAEDT INFO: Deleted 1 Objects: right_winding_cross.
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Union of 3 objects has been executed.


### 修改设置
可以直接用`m3d['variable']=new value`的方式来直接修改

In [5]:
m3d['_r'] = '6.5mm'
m3d['_k'] = '600percent'
m3d['_w'] = '11mm'

### 添加激励

In [5]:
m3d.assign_current(
    ['left_winding_cross'],
    amplitude = '1A',
    swap_direction = False,
    name = 'left_winding_current'
)

PyAEDT ERROR: API Message - Project: Project2, Design: Maxwell3DDesign1 (EddyCurrent), [error] Script macro error: Duplicate excitation name: 'left_winding_current' (2:44:33 PM  Jul 04, 2023)

PyAEDT ERROR:   File "c:\Users\silergy\AppData\Local\Programs\Python\Python38\lib\site-packages\pyaedt\modules\Boundary.py", line 479, in create, GENERAL OR AEDT ERROR
PyAEDT ERROR: Check Online documentation on: https://aedt.docs.pyansys.com/version/stable/search.html?q=create


False

### 开始仿真

In [6]:
m3d.analyze_all()

True

## 在ansys中添加绘图
这里以绘制bottom_plate中的B场幅值为例

In [7]:
# 在volume上添加B场幅值
Ansys.create_field_plot_volume(m3d, "bottom_plate", 'Mag_B', 'MagB in bottom plate')

更高阶的功能可以直接通过操作pyaedt的API来实现如下所示

In [13]:
quantity_name2 = "Mag_B"
setup_name = "Setup1"
intrinsic = {"Freq": "1MHz", "Phase": "0deg"}
vollist = ["Top_plate_bottom"]

# 在某一个面上绘制B场幅值
plot2 = m3d.post.create_fieldplot_surface(m3d.modeler["Top_plate"].faces[1].id, 'Mag_B', m3d.existing_analysis_sweeps[0], intrinsic)
# 导出图片
m3d.post.export_field_jpg(
    'E:\\Desktop\\Power_toys\\Power_toys\\prova2.jpg',
    plot2.name,
    plot2.plotFolder,
    orientation='top',
    display_wireframe=False,
)
# 导出数据到文件
m3d.post.export_field_file_on_grid(
    quantity_name="Mag_B",
    solution="Setup1 : LastAdaptive",
    filename='E:\\Desktop\\Power_toys\\Power_toys\\Mag.csv',
    grid_start=[6,6,-0.2],
    grid_stop=[6, 7, -0.2],
    grid_step=[0.1, 0.1, 0],
    intrinsics="1MHz",
)

PyAEDT INFO: Exporting Mag_B field. Be patient


'E:\\Desktop\\Power_toys\\Power_toys\\Mag.csv'